In [1]:
import gerry_v2 as gerry

from importlib import reload; reload(gerry);

import numpy as np
import pandas as pd 
import csv
from glob import glob

### Database

In [2]:
# Instantiate a GerryDB object
g = gerry.GerryDB()

In [3]:
# Create the tPlan table, to store all the simulated plans
g.CreatePlanTable()

In [4]:
g.CreateScoresTable()

In [5]:
g.CreateVTDTable()

In [6]:
g.RunQuery("SELECT * FROM tPlan;")

,plan_id,vtd,iter,dist


In [7]:
g.RunQuery("SELECT * FROM tScores;")

,plan_id,dist,seg_score,bvap


In [8]:
g.RunQuery("SELECT * FROM tVtd;")

,vtd,incumbent,geography


### Loading Data?

In [9]:
PATH = '../Output/Chain_02.23/next/'
PATH_2 = '../Output/Chain_02.23/used/'

In [15]:
files = glob(PATH + '*') + glob(PATH_2 + '*')

In [16]:
file = files[0]

In [10]:
def prep(file):

    seed = file.split('_')[3]

    data = pd.read_csv(file)
    data['Iteration'] = [x[:-6] for x in data['Iteration']]
    data.rename(columns={'Iteration':'vtd'}, inplace=True)

    data_m = data.melt(id_vars='vtd',var_name='iter',value_name='dist')
    data_m["plan_id"] = seed + '_' + data_m["iter"]

    return(data_m)

In [17]:
df = prep(file)

In [18]:
df

,vtd,iter,dist,plan_id
0,01003000001,1,1003,1.2.5.8.19.39.124.177.414.736_1
1,01003000002,1,1003,1.2.5.8.19.39.124.177.414.736_1
2,01003000003,1,1003,1.2.5.8.19.39.124.177.414.736_1
3,01003000004,1,1003,1.2.5.8.19.39.124.177.414.736_1
4,01003000005,1,1003,1.2.5.8.19.39.124.177.414.736_1
...,...,...,...,...
3687995,01131005006,2000,1005,1.2.5.8.19.39.124.177.414.736_2000
3687996,01131006001,2000,1005,1.2.5.8.19.39.124.177.414.736_2000
3687997,01131006002,2000,1005,1.2.5.8.19.39.124.177.414.736_2000
3687998,01131006004,2000,1005,1.2.5.8.19.39.124.177.414.736_2000


In [38]:
df[df.duplicated()]

,vtd,iter,dist,plan_id
425,01053000120,1,1005,1.2.5.8.19.39.124.177.414.736_1
1185,01077000007,1,1002,1.2.5.8.19.39.124.177.414.736_1
1352,01021000120,1,1007,1.2.5.8.19.39.124.177.414.736_1
1637,01073002180,1,1004,1.2.5.8.19.39.124.177.414.736_1
1675,01073003210,1,1004,1.2.5.8.19.39.124.177.414.736_1
...,...,...,...,...
3687508,01021000120,2000,1005,1.2.5.8.19.39.124.177.414.736_2000
3687793,01073002180,2000,1002,1.2.5.8.19.39.124.177.414.736_2000
3687831,01073003210,2000,1002,1.2.5.8.19.39.124.177.414.736_2000
3687874,01101000104,2000,1004,1.2.5.8.19.39.124.177.414.736_2000


In [19]:
df.iloc[0]

vtd                            01003000001
iter                                     1
dist                                  1003
plan_id    1.2.5.8.19.39.124.177.414.736_1
Name: 0, dtype: object

In [20]:
sql = "INSERT INTO tPlan" + " (" + \
           ",".join([c for c in df.columns]) + \
           ") VALUES (" + ",".join([':' + c for c in df.columns]) + ");"

In [21]:
diction = df.to_dict(orient='records')

In [22]:
diction[0]

{'vtd': '01003000001',
 'iter': '1',
 'dist': 1003,
 'plan_id': '1.2.5.8.19.39.124.177.414.736_1'}

In [30]:
g.Connect()

g.curs.execute(sql, diction[3])

g.conn.commit()
g.conn.close()

In [32]:
for row in diction:
    print(row)
    break

{'vtd': '01003000001', 'iter': '1', 'dist': 1003, 'plan_id': '1.2.5.8.19.39.124.177.414.736_1'}


In [36]:
g.RunQuery("""SELECT * FROM tPlan
           WHERE vtd = '01101000104'
           AND iter = 2000;""")

,plan_id,vtd,iter,dist
0,1.2.5.8.19.39.124.177.414.736_2000,01101000104,2000,1004


In [ ]:
g.Connect()

for row in df.to_dict(orient='records'):
        try:
            g.curs.execute(sql,row)
        except Exception as e:
            print(e)
            print(row)

g.conn.commit()
g.conn.close()


### Duplicates?

In [59]:
df_2 = pd.read_csv(file)
df_2['Iteration'] = [x[:-6] for x in df_2['Iteration']]
df_2.rename(columns={'Iteration':'vtd'}, inplace=True)
df_2_m = df_2.melt(id_vars='vtd',var_name='iter',value_name='dist')

In [61]:
df_2_m[df_2_m.duplicated()]

,vtd,iter,dist
425,01053000120,1,1005
1185,01077000007,1,1002
1352,01021000120,1,1007
1637,01073002180,1,1004
1675,01073003210,1,1004
...,...,...,...
3687508,01021000120,2000,1005
3687793,01073002180,2000,1002
3687831,01073003210,2000,1002
3687874,01101000104,2000,1004


In [62]:
idx = df_2_m['vtd'] == '01053000120'

In [64]:
df_2_m[idx]

,vtd,iter,dist
56,01053000120,1,1005
425,01053000120,1,1005
1900,01053000120,2,1005
2269,01053000120,2,1005
3744,01053000120,3,1005
...,...,...,...
3682893,01053000120,1998,1005
3684368,01053000120,1999,1005
3684737,01053000120,1999,1005
3686212,01053000120,2000,1005


In [67]:
df_2 = pd.read_csv(file)
df_2.rename(columns={'Iteration':'vtd'}, inplace=True)
df_2_m = df_2.melt(id_vars='vtd',var_name='iter',value_name='dist')

In [70]:
df_2_m[df_2_m['vtd'].str.startswith('01053000120', na=False)]

,vtd,iter,dist
56,01053000120_01001,1,1005
425,01053000120_01002,1,1005
1900,01053000120_01001,2,1005
2269,01053000120_01002,2,1005
3744,01053000120_01001,3,1005
...,...,...,...
3682893,01053000120_01002,1998,1005
3684368,01053000120_01001,1999,1005
3684737,01053000120_01002,1999,1005
3686212,01053000120_01001,2000,1005


In [75]:
df_2_m

,vtd,iter,dist
0,01003000001_01001,1,1003
1,01003000002_01001,1,1003
2,01003000003_01001,1,1003
3,01003000004_01001,1,1003
4,01003000005_01001,1,1003
...,...,...,...
3687995,01131005006_01007,2000,1005
3687996,01131006001_01007,2000,1005
3687997,01131006002_01007,2000,1005
3687998,01131006004_01007,2000,1005


In [82]:
df_2[df_2['vtd'].str.startswith('01053000120', na=False)]

,vtd,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
56,01053000120_01001,1005,1005,1005,1003,1003,1005,1005,1005,1005,...,1005,1005,1007,1007,1005,1005,1005,1005,1005,1005
425,01053000120_01002,1005,1005,1005,1003,1003,1005,1005,1005,1005,...,1005,1005,1007,1007,1005,1005,1005,1005,1005,1005


In [74]:
df_2[df_2['vtd'].duplicated()]

,vtd,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000


In [58]:
prep(file)[prep(file).duplicated()]

,vtd,iter,dist,plan_id
425,01053000120,1,1005,1.2.5.8.19.39.124.177.414.736_1
1185,01077000007,1,1002,1.2.5.8.19.39.124.177.414.736_1
1352,01021000120,1,1007,1.2.5.8.19.39.124.177.414.736_1
1637,01073002180,1,1004,1.2.5.8.19.39.124.177.414.736_1
1675,01073003210,1,1004,1.2.5.8.19.39.124.177.414.736_1
...,...,...,...,...
3687508,01021000120,2000,1005,1.2.5.8.19.39.124.177.414.736_2000
3687793,01073002180,2000,1002,1.2.5.8.19.39.124.177.414.736_2000
3687831,01073003210,2000,1002,1.2.5.8.19.39.124.177.414.736_2000
3687874,01101000104,2000,1004,1.2.5.8.19.39.124.177.414.736_2000
